In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [84]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# Read the CSV and Perform Basic Data Cleaning

In [4]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [52]:
# Load the data
file_path = Path('../Resources/LoanStats_2019Q1.csv.zip')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)
pd.set_option('display.max_columns', None)
df.tail()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,next_pymnt_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
68812,10000.0,0.1502,346.76,RENT,26000.0,Source Verified,Jan-2019,low_risk,n,9.60,0.0,0.0,9.0,0.0,2684.0,20.0,w,9326.86,9326.86,1019.42,1019.42,673.14,346.28,0.0,0.0,0.0,346.76,May-2019,0.0,1.0,Individual,0.0,242.0,6798.0,1.0,3.0,0.0,0.0,160.0,4114.0,76.0,3.0,6.0,977.0,33.0,15200.0,0.0,1.0,1.0,6.0,755.0,9677.0,14.4,0.0,0.0,185.0,17.0,4.0,4.0,0.0,4.0,3.0,4.0,3.0,5.0,3.0,3.0,14.0,6.0,6.0,5.0,9.0,0.0,0.0,0.0,3.0,80.0,0.0,0.0,0.0,20625.0,6798.0,11300.0,5425.0,N,N
68813,12000.0,0.2727,368.37,RENT,63000.0,Not Verified,Jan-2019,low_risk,n,29.07,0.0,0.0,8.0,0.0,13314.0,26.0,w,11706.42,11706.42,1086.93,1086.93,293.58,793.35,0.0,0.0,0.0,368.37,May-2019,0.0,1.0,Individual,0.0,0.0,60350.0,3.0,4.0,2.0,2.0,2.0,47036.0,75.0,3.0,5.0,1177.0,69.0,25000.0,1.0,0.0,4.0,7.0,7544.0,11387.0,15.7,0.0,0.0,164.0,107.0,1.0,1.0,0.0,1.0,2.0,0.0,3.0,4.0,3.0,4.0,18.0,4.0,8.0,4.0,8.0,0.0,0.0,0.0,5.0,96.2,0.0,0.0,0.0,87939.0,60350.0,13500.0,62939.0,N,N
68814,5000.0,0.1992,185.62,MORTGAGE,52000.0,Source Verified,Jan-2019,low_risk,n,14.86,0.0,0.0,5.0,1.0,3715.0,14.0,w,4666.78,4666.78,571.33,571.33,333.22,238.11,0.0,0.0,0.0,185.62,May-2019,0.0,1.0,Individual,0.0,0.0,18611.0,2.0,1.0,1.0,1.0,8.0,14896.0,81.0,2.0,3.0,0.0,61.0,12100.0,0.0,2.0,1.0,4.0,3722.0,3600.0,0.0,0.0,0.0,197.0,91.0,0.0,0.0,1.0,0.0,10.0,0.0,0.0,2.0,2.0,2.0,8.0,4.0,5.0,2.0,5.0,0.0,0.0,0.0,3.0,100.0,0.0,1.0,0.0,30592.0,18611.0,3600.0,18492.0,N,N
68815,40000.0,0.0646,1225.24,MORTGAGE,520000.0,Verified,Jan-2019,low_risk,n,9.96,0.0,1.0,21.0,0.0,59529.0,57.0,f,36953.94,36953.94,3661.36,3661.36,3046.06,615.30,0.0,0.0,0.0,1225.24,May-2019,0.0,1.0,Individual,0.0,0.0,864480.0,2.0,3.0,0.0,0.0,28.0,27151.0,35.0,3.0,5.0,38479.0,34.0,111100.0,1.0,0.0,3.0,7.0,41166.0,41467.0,34.3,0.0,0.0,147.0,146.0,2.0,2.0,9.0,3.0,3.0,0.0,4.0,5.0,8.0,16.0,10.0,14.0,36.0,5.0,21.0,0.0,0.0,0.0,3.0,98.2,12.5,0.0,0.0,1033574.0,95958.0,100800.0,78634.0,N,N
68816,16000.0,0.1131,350.36,MORTGAGE,72000.0,Verified,Jan-2019,low_risk,n,7.02,2.0,0.0,12.0,1.0,11882.0,39.0,w,15395.66,15395.66,1041.03,1041.03,604.34,436.69,0.0,0.0,0.0,350.36,May-2019,0.0,1.0,Individual,0.0,0.0,225413.0,0.0,2.0,0.0,0.0,28.0,62953.0,79.0,0.0,0.0,5568.0,45.0,32000.0,2.0,2.0,4.0,1.0,18784.0,11705.0,49.1,2.0,0.0,159.0,129.0,25.0,24.0,3.0,25.0,1.0,2.0,4.0,5.0,5.0,9.0,20.0,9.0,16.0,5.0,12.0,0.0,0.0,2.0,0.0,94.3,0.0,1.0,0.0,251486.0,74835.0,23000.0,63090.0,N,N


# Split the Data into Training and Testing

In [72]:
# Create our target
y = df.copy()
y.reset_index(inplace=True)
# Create our features
X = df.drop(columns=['loan_status','issue_d','next_pymnt_d'])


In [73]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,policy_code,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0,68817.000000,68817.000000,68817.0,68817.0,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.00000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,24.666347,16018.845833,16015.919519,970.639473,970.431151,658.748750,311.855687,0.035072,0.0,0.0,517.202218,0.016188,1.0,0.0,188.664298,1.634976e+05,1.023047,3.290481,0.843730,1.909644,14.303152,4.324044e+04,69.21191,1.240042,2.622245,6297.491710,55.702472,42431.895360,1.382071,1.687795,2.189023,4.827572,14142.936193,16991.852667,48.330667,0.006830,2.347065,125.468605,175.136841,14.294215,7.179781,1.441882,24.714954,7.688028,0.472485,3.806123,5.648038,5.135156,7.414403,9.500153,8.630818,13.517663,5.653036,12.571995,0.0,0.0,0.052138,2.219423,95.057627,30.626217,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,12.318917,10002.695495,10004.985940,1229.563301,1229.569864,1126.237377,279.786954,1.086720,0.0,0.0,816.021053,0.134774,0.0,0.0,1808.457195,1.723699e+05,1.176782,2.965579,0.970543,1.544994,13.972619,4.956130e+04,22.33711,1.471745,2.502608,5702.281757,18.495480,37166.229316,1.543220,2.852664,2.396186,3.145377,15863.878993,20447.276820,28.106287,0.087327,301.312407,56.968853,99.149467,17.417118,6.303066,1.747777,32.130889,6.034861,1.410819,2.464504,3.470941,3.230651,4.581371,7.573846,5.056994,8.026158,3.472679,6.021318,0.0,0.0,0.390633,1.897432,8.326426,33.631463,0.336732,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,1.0,0.0,0.000000,2.350000e+02,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000e+00,0.00000,0.000000,0.000000,0.000000,1.000000,200.000000,0.000000,0.000000,0.000000,0.000000,20.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,0.000000,0.000000,0.

In [88]:
# Check the balance of our target values
y['loan_status'].value_counts()

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [89]:
y = df["loan_status"].values.reshape(-1, 1)

In [90]:
X_encoded = pd.get_dummies(X, columns=["application_type","initial_list_status","policy_code","home_ownership", "verification_status","pymnt_plan","hardship_flag","debt_settlement_flag"])

In [77]:
# Split the X and y into X_train, X_test, y_train, y_test
# YOUR CODE HERE

X_train, X_test, y_train, y_test = train_test_split(X_encoded, 
                                                    y, 
                                                    random_state=1, 
                                                    )

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [91]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [92]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
# YOUR CODE HERE
scaler.fit(X_train)

StandardScaler()

In [93]:
# Scale the training and testing data
# YOUR CODE HERE
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [113]:
# Resample the training data with the BalancedRandomForestClassifier
# YOUR CODE HERE
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=500, random_state=78)
rf_model = rf_model.fit(X_train, y_train)

In [82]:
predictions = rf_model.predict(X_test)

In [99]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
acc_score = balanced_accuracy_score(y_test.values.argmax(axis=1), predictions.argmax(axis=1))
print(acc_score)

0.6483102209893581


In [98]:
# Display the confusion matrix
# YOUR CODE HERE
confusion_matrix(y_test.values.argmax(axis=1), predictions.argmax(axis=1))

array([[   30,    71],
       [    7, 17097]], dtype=int64)

In [103]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test.values.argmax(axis=1), predictions.argmax(axis=1)))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.81      0.30      1.00      0.43      0.54      0.28       101
          1       1.00      1.00      0.30      1.00      0.54      0.32     17104

avg / total       0.99      1.00      0.30      0.99      0.54      0.32     17205



In [107]:
# List the features sorted in descending order by feature importance
# YOUR CODE HERE
importances = rf_model.feature_importances_
importances_sorted = sorted(zip(rf_model.feature_importances_, X.columns), reverse= True)
importances_sorted

[(0.06724857550434013, 'out_prncp_inv'),
 (0.06456053652365182, 'initial_list_status'),
 (0.06417899546010511, 'out_prncp'),
 (0.058464794243587956, 'total_acc'),
 (0.05676774161684017, 'total_rec_int'),
 (0.019630104534277024, 'annual_inc'),
 (0.018267343117629134, 'open_il_24m'),
 (0.01752530633531314, 'avg_cur_bal'),
 (0.01689076547925234, 'revol_bal'),
 (0.01683156266419943, 'pub_rec'),
 (0.016796528142761724, 'inq_last_6mths'),
 (0.01664125867372644, 'total_rev_hi_lim'),
 (0.01656701286431407, 'num_tl_op_past_12m'),
 (0.01639447210241601, 'num_tl_90g_dpd_24m'),
 (0.016214916355622746, 'acc_open_past_24mths'),
 (0.015677479580162863, 'installment'),
 (0.015482433589671284, 'inq_fi'),
 (0.01545295294713466, 'tot_cur_bal'),
 (0.014929296421432637, 'percent_bc_gt_75'),
 (0.01489208485733072, 'all_util'),
 (0.014879683225887558, 'last_pymnt_amnt'),
 (0.014800768275190898, 'pct_tl_nvr_dlq'),
 (0.014720253477270313, 'home_ownership'),
 (0.013659728206834993, 'total_bal_il'),
 (0.01320842

### Easy Ensemble Classifier

In [116]:
# Train the EasyEnsembleClassifier
# YOUR CODE HERE
from imblearn.ensemble import EasyEnsembleClassifier
ee_model = EasyEnsembleClassifier(base_estimator=None, n_estimators=100, n_jobs=1,
            random_state=1, replacement=False, sampling_strategy='auto',
            verbose=0, warm_start=False)
ee_model = ee_model.fit(X_train, y_train.values.argmax(axis=1))

In [117]:
#Make Predictions Using EE
ee_predictions = ee_model.predict(X_test)

In [120]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
ee_acc_score = balanced_accuracy_score(y_test.values.argmax(axis=1), ee_predictions)
print(ee_acc_score)

0.7339687780751882


In [122]:
# Display the confusion matrix
# YOUR CODE HERE
confusion_matrix(y_test.values.argmax(axis=1), ee_predictions)

array([[   71,    30],
       [ 4020, 13084]], dtype=int64)

In [124]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test.values.argmax(axis=1), ee_predictions))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.02      0.70      0.76      0.03      0.73      0.53       101
          1       1.00      0.76      0.70      0.87      0.73      0.54     17104

avg / total       0.99      0.76      0.70      0.86      0.73      0.54     17205

